In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Ossora,59.2353,163.0719,35.08,overcast clouds,86,100,10.92,RU
1,1,Ateptsevo,55.3236,36.7500,57.61,broken clouds,81,84,7.09,RU
2,2,Babushkin,51.7122,105.8647,45.46,clear sky,76,3,6.44,RU
3,3,Tupelo,34.2576,-88.7034,72.99,broken clouds,60,75,13.80,US
4,4,Rikitea,-23.1203,-134.9692,77.47,broken clouds,83,67,11.03,PF


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temp you would like to have for your vacation?"))
max_temp = float(input("What is the maximum temp you would like to have for your vacation?"))

What is the minimum temp you would like to have for your vacation?65
What is the maximum temp you would like to have for your vacation?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
destination_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
destination_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
3,3,Tupelo,34.2576,-88.7034,72.99,broken clouds,60,75,13.80,US
4,4,Rikitea,-23.1203,-134.9692,77.47,broken clouds,83,67,11.03,PF
5,5,Sucua,-2.4667,-78.1667,80.76,light rain,71,75,4.23,EC
7,7,Kapaa,22.0752,-159.3190,75.18,overcast clouds,83,96,5.01,US
9,9,Saint-Philippe,-21.3585,55.7679,72.99,scattered clouds,83,40,12.68,RE
10,10,Namatanai,-3.6667,152.4333,79.34,broken clouds,84,62,8.16,PG
13,13,Salalah,17.0151,54.0924,81.14,clear sky,80,2,6.71,OM
16,16,Stroud,51.7500,-2.2000,66.16,scattered clouds,50,33,4.85,GB
17,17,Newport,51.5877,-2.9984,66.31,broken clouds,66,81,3.44,GB
26,26,Castro,-24.7911,-50.0119,66.15,scattered clouds,46,35,4.56,BR


In [5]:
# 4a. Determine if there are any empty rows.
destination_cities_df.count()

City_ID                307
City                   307
Lat                    307
Lng                    307
Max Temp               307
Current Description    307
Humidity               307
Cloudiness             307
Wind Speed             307
Country                307
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_destination_cities = destination_cities_df.dropna()
clean_destination_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
3,3,Tupelo,34.2576,-88.7034,72.99,broken clouds,60,75,13.80,US
4,4,Rikitea,-23.1203,-134.9692,77.47,broken clouds,83,67,11.03,PF
5,5,Sucua,-2.4667,-78.1667,80.76,light rain,71,75,4.23,EC
7,7,Kapaa,22.0752,-159.3190,75.18,overcast clouds,83,96,5.01,US
9,9,Saint-Philippe,-21.3585,55.7679,72.99,scattered clouds,83,40,12.68,RE


In [7]:
clean_destination_cities.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Current Description',
       'Humidity', 'Cloudiness', 'Wind Speed', 'Country'],
      dtype='object')

In [9]:

# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_destination_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tupelo,US,72.99,broken clouds,34.2576,-88.7034,
4,Rikitea,PF,77.47,broken clouds,-23.1203,-134.9692,
5,Sucua,EC,80.76,light rain,-2.4667,-78.1667,
7,Kapaa,US,75.18,overcast clouds,22.0752,-159.3190,
9,Saint-Philippe,RE,72.99,scattered clouds,-21.3585,55.7679,
10,Namatanai,PG,79.34,broken clouds,-3.6667,152.4333,
13,Salalah,OM,81.14,clear sky,17.0151,54.0924,
16,Stroud,GB,66.16,scattered clouds,51.7500,-2.2000,
17,Newport,GB,66.31,broken clouds,51.5877,-2.9984,
26,Castro,BR,66.15,scattered clouds,-24.7911,-50.0119,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Not Found, Skip City")

Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City
Not Found, Skip City


In [21]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tupelo,US,72.99,broken clouds,34.2576,-88.7034,"Hilton Garden Inn Tupelo, MS"
4,Rikitea,PF,77.47,broken clouds,-23.1203,-134.9692,People ThankYou
5,Sucua,EC,80.76,light rain,-2.4667,-78.1667,Hotel Orellana
7,Kapaa,US,75.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Saint-Philippe,RE,72.99,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tupelo,US,72.99,broken clouds,34.2576,-88.7034,"Hilton Garden Inn Tupelo, MS"
4,Rikitea,PF,77.47,broken clouds,-23.1203,-134.9692,People ThankYou
5,Sucua,EC,80.76,light rain,-2.4667,-78.1667,Hotel Orellana
7,Kapaa,US,75.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Saint-Philippe,RE,72.99,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))